# Лабораторная работа №5

## Тайные многосторонние вычисления и разделение секрета

**Задание:**

Необходимо привести последовательность выполнения следующих протоколов:
- тайных многосторонних вычислений для расчета средней величины трех чисел. В качестве исходных данных принять коды 1-ой, 2-ой и 3-ей буквы своей фамилии согласно их положению в алфавите;
- разбиения секрета с использованием гаммирования для трех участников. В качестве секрета принять первые 3 буквы фамилии, для гамм - любые трехбуквенные сочетания;
- разделения секрета по схеме Шамира для (3, 5)-пороговой схемы. В качестве секрета S принять код 1-ой буквы своей фамилии согласно ее положению в алфавите;
- разделения секрета по схеме Асмута-Блума для (3, 5)-пороговой схемы. В качестве секрета S принять код 1-ой буквы своей фамилии согласно ее положению в алфавите

### Импорт необходимых библиотек и модулей

In [1]:
import numpy as np

In [2]:
from core.params_generator import generate_secret_multilateral_protocol_parameters
from core.utils import *
from core.protocols import *

Для воспроизводимости экспериментов

In [3]:
init_random_seed(893)

### Протокол тайных многосторонних вычислений для расчета средней величины трех чисел 

В качестве кодов возьмем номера первых трех букв фамилии по алфавиту

In [4]:
letters = ['Е', 'л', 'и']
codes = [letter_to_code(letter) for letter in letters]
codes

[6, 13, 10]

Сгенерируем параметры шифрования необходимые для работы алгоритма. Для каждого "агента" это будут:

- Открытый ключ e;
- Открытый ключ n;
- Закрытый ключ

In [5]:
encryption_params = []

for i, param in enumerate(generate_secret_multilateral_protocol_parameters()):
        encryption_params.append({
        'code': codes[i],
        'n': param['n'],
        'e': param['e'],
        'd': param['d']
    })
    
encryption_params

[{'code': 6, 'n': 91, 'e': 7, 'd': 31},
 {'code': 13, 'n': 91, 'e': 5, 'd': 29},
 {'code': 10, 'n': 91, 'e': 17, 'd': 17}]

Вычислим среднее значение, используя протокол тайных многосторонних вычислений

In [6]:
average_value = secret_multilateral_protocol(encryption_params)
print(f"Среднее значение: {average_value}")

Шифрование агентом №1:

x = 32
(6 + 32) ** 5 mod 91 = 12
Шифрование агентом №2:

x = 12 ** 29 % 91 = 38
(13 + 38) ** 17 mod 91 = 25

Шифрование агентом №3:

x = 25 ** 17 % 91 = 51
(10 + 51) ** 7 mod 91 = 61

Расшифровка результата:

61 ** 31 % 91 = 61
(61 - 32) / 3 = 9.666666666666666

Среднее значение: 9.666666666666666


Проверим корректность работы протокола

In [7]:
assert np.mean(codes) == average_value
print("Протокол тайных многосторонних вычислений для расчета средней величины трех чисел выполнен успешно")

Протокол тайных многосторонних вычислений для расчета средней величины трех чисел выполнен успешно


### Протокол разбиения секрета с использованием гаммирования

В качестве секрета возьмем первые 3 буквы фамилии, представленные списком символов в верхнем регистре

In [8]:
secret = ['Е', 'Л', 'И']

Запустим работу протокола

In [9]:
decrypted, codes = gamma_secret_split_protocol(secret, n_gammas=3)

Исходные коды:

['11000101', '11001011', '11001000']

Гаммы:

[['Щ', 'Ж', 'Х'], ['О', 'Ф', 'Х'], ['В', 'Ь', 'О']]

Гаммы в закодированном виде:

[['11011001', '11000110', '11010101'], ['11001110', '11010100', '11010101'], ['11000010', '11011100', '11001110']]

Коды шифрования

[['11000101', '11001011', '11001000'], ['11011001', '11000110', '11010101'], ['11001110', '11010100', '11010101'], ['11000010', '11011100', '11001110']]

Зашифрованные коды:

['00010000', '00000101', '00000110']

Коды дешифрования

[['11011001', '11000110', '11010101'], ['11001110', '11010100', '11010101'], ['11000010', '11011100', '11001110'], ['00010000', '00000101', '00000110']]

Дешифрованные коды:

['11000101', '11001011', '11001000']


Проверим корректность работы протокола

In [10]:
assert decrypted == codes
print("Протокол разбиения секрета с использованием гаммирования выполнен успешно")

Протокол разбиения секрета с использованием гаммирования выполнен успешно


### Протокол разделения секрета по схеме Шамира

В качестве секрета возьмем первую букву фамилии, представленную в Unicode

In [11]:
secret = ord('Е')
print(f"Зашифрованный код: {secret}")

Зашифрованный код: 1045


Определим параметры системы:

- число долей разбиения;
- минимальное необходимое число долей для восстановления;
- простое число, превосходящее секрет

In [12]:
n = 5
m = 3
p = 2089

Проведем разбиение по схеме Шамира

In [13]:
shares = shamir_split_protocol(secret, n, m, p)

Коэффициенты полинома:

[1045, 795, 1828]

Сгенерированные доли:

[(1, 1579), (2, 1591), (3, 1081), (4, 49), (5, 584)]


По сгенерированным долям восстановим секрет

In [14]:
recovered_secret = shamir_recover(shares, p)

Используем доли для восстановления секрета:

[(1, 1579), (2, 1591), (3, 1081), (4, 49), (5, 584)]

Процесс интерполяции для x = 0:
  Применение к точке (1, 1579):
    Умножаем на (x - 2) / (1 - 2)...
    Умножаем на (x - 3) / (1 - 3)...
    Умножаем на (x - 4) / (1 - 4)...
    Умножаем на (x - 5) / (1 - 5)...
  Промежуточная сумма для точки 0: 1628
  Применение к точке (2, 1591):
    Умножаем на (x - 1) / (2 - 1)...
    Умножаем на (x - 3) / (2 - 3)...
    Умножаем на (x - 4) / (2 - 4)...
    Умножаем на (x - 5) / (2 - 5)...
  Промежуточная сумма для точки 1: 341
  Применение к точке (3, 1081):
    Умножаем на (x - 1) / (3 - 1)...
    Умножаем на (x - 2) / (3 - 2)...
    Умножаем на (x - 4) / (3 - 4)...
    Умножаем на (x - 5) / (3 - 5)...
  Промежуточная сумма для точки 2: 706
  Применение к точке (4, 49):
    Умножаем на (x - 1) / (4 - 1)...
    Умножаем на (x - 2) / (4 - 2)...
    Умножаем на (x - 3) / (4 - 3)...
    Умножаем на (x - 5) / (4 - 5)...
  Промежуточная сумма для точки 3

In [15]:
print(f"Восстановленный код: {recovered_secret}")

Восстановленный код: 1045


Проверим корректность работы протокола

In [16]:
assert recovered_secret == secret
print("Протокол разделения секрета по схеме Шамира выполнен успешно")

Протокол разделения секрета по схеме Шамира выполнен успешно


### Протокол разделения секрета по схеме Асмута-Блума

Используем те же параметры протокола, что и в случае схемы Шамира

Сгенерируем доли

In [17]:
d, shares = secret_sharing_scheme(secret, n, m, p)

Таблица долей (di, ki):
(2089, 1045)
(2099, 1045)
(2111, 1045)
(2113, 1045)
(2129, 1045)

Публикуем p: 2089


По сгенерированным долям восстановим секрет с помощью [китайской теоремы об остатках](https://ru.wikipedia.org/wiki/Китайская_теорема_об_остатках)

In [18]:
recovered_secret = secret_recovery(d[:m], shares[:m])
print(f"Восстановленный секрет: {recovered_secret}")


Восстановление секрета с помощью китайской теоремы об остатках...

Восстановленный секрет: 1045


Проверим корректность работы протокола

In [19]:
assert recovered_secret == secret
print("Протокол разделения секрета по схеме Асмута-Блума выполнен успешно")

Протокол разделения секрета по схеме Асмута-Блума выполнен успешно
